In [94]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [95]:
# Read data from datasource
def read_csv_file(file_path):
    try:
        # Attempt to read the CSV file
        data = pd.read_csv(file_path)
        print("File read successfully!")
        return data
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
    except pd.errors.EmptyDataError:
        print("Error: The file is empty.")
    except pd.errors.ParserError:
        print("Error: The file contains parsing errors.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Call read data func
file_path = '../data/HomeTask_ [1] User Information Labelled.csv'
labelled_user_info_df = read_csv_file(file_path)

if labelled_user_info_df is not None:
    print(labelled_user_info_df.shape)  # Display dataframe shape if data was successfully loaded


File read successfully!
(500, 16)


In [96]:
# Drop duplicate rows from 'labelled_user_info_df'
labelled_user_info_df = labelled_user_info_df.drop_duplicates()
print(labelled_user_info_df.shape)

(455, 16)


In [97]:
# Remove Missing data records
def remove_nan_rows(df, id_column):
    # Step 1: Remove rows with NaN values, but only if there are other rows with valid data for the same 'user_id'
    df_cleaned = df.dropna(subset=df.columns.difference([id_column]))
    
    # Step 2: Return the cleaned DataFrame
    return df_cleaned

# Call remove_nan_rows
cleaned_user_info_df = remove_nan_rows(labelled_user_info_df, 'user_id')
print(cleaned_user_info_df.shape)

(432, 16)


In [98]:
# Call read data func
file_path = '../data/HomeTask_ [2] Transaction Data Labelled.csv'
labelled_transaction_df = read_csv_file(file_path)

if labelled_transaction_df is not None:
    print(labelled_transaction_df.shape)  # Display shape of dataframe if data was successfully loaded

File read successfully!
(5945, 11)


In [99]:
# Merge both datasets on 'user_id'
df = pd.merge(cleaned_user_info_df, labelled_transaction_df, on='user_id')
df.shape

(6622, 26)

In [100]:
df.head()

,user_id,session_id,country_of_residence,has_biometrics,phone_number_country_code,reason_to_use_app,occupation,kyc_state,gender,date_of_birth,...,id,reference_transaction_id,state,type,category_id,amount,currency,direction,completed_at,created_at
0,005256f3-53d4-471c-8bbd-b0c5e16bf2f7,882dd5b7-e378-4947-89e0-900a79b3083f,FR,True,33,TRAVEL_ACCOUNT,SERVICE_AND_SALES,COMPLETED,M,2002-03-09,...,be097278-6ce9-4afe-84cc-96e24ed59e44,62fe1b69-52a9-4ae6-b13e-2d7932fc8d12,PENDING,PROCESSING_FEE,b818b29c-cd13-4352-863d-f930de135c65,14.99,EUR,OUT,NaN,2024-08-04 15:26:52.418093 UTC
1,005256f3-53d4-471c-8bbd-b0c5e16bf2f7,882dd5b7-e378-4947-89e0-900a79b3083f,FR,True,33,TRAVEL_ACCOUNT,SERVICE_AND_SALES,COMPLETED,M,2002-03-09,...,d0bb6764-6302-4444-94e9-0024302c2f90,e9ab418d-b2a7-46fe-8693-7d6c9110d1bb,COMPLETED,BANK_TRANSFER,e464e1e2-40ca-416d-a541-ba0865e252f9,5.00,EUR,IN,2024-08-06 12:32:23.350767 UTC,2024-08-06 12:32:14.305982 UTC
2,005256f3-53d4-471c-8bbd-b0c5e16bf2f7,882dd5b7-e378-4947-89e0-900a79b3083f,FR,True,33,TRAVEL_ACCOUNT,SERVICE_AND_SALES,COMPLETED,M,2002-03-09,...,ab000ca3-28b3-4610-8253-2b4f32d12da3,910b847b-c944-4cc6-8857-ba75d6befa46,DECLINED,CARDS,06bee557-499a-4d75-bc66-155ea0489a19,2.50,EUR,OUT,2024-08-20 11:05:34.973154 UTC,2024-08-20 11:05:34.973154 UTC
3,005256f3-53d4-471c-8bbd-b0c5e16bf2f7,882dd5b7-e378-4947-89e0-900a79b3083f,FR,True,33,TRAVEL_ACCOUNT,SERVICE_AND_SALES,COMPLETED,M,2002-03-09,...,dcc483c5-227e-4585-a874-6a0552ad771f,67859c37-b0e4-44b4-98b9-d53d66c97c42,COMPLETED,MANUAL,b818b29c-cd13-4352-863d-f930de135c65,25.01,EUR,OUT,2024-08-20 08:22:31.556082 UTC,2024-08-20 08:22:31.556082 UTC
4,005256f3-53d4-471c-8bbd-b0c5e16bf2f7,882dd5b7-e378-4947-89e0-900a79b3083f,FR,True,33,TRAVEL_ACCOUNT,SERVICE_AND_SALES,COMPLETED,M,2002-03-09,...,0aa0343d-4db8-4c93-b0ba-bc01e54befd8,03baa814-7bc3-4fdd-9f7f-580a5a76bf71,DECLINED,CARDS,8c6dc9fc-af02-4754-a312-25d7d05c091b,11.73,EUR,OUT,2024-08-13 13:07:10.712891 UTC,2024-08-13 13:07:10.712891 UTC


In [101]:
nan_control_cols = ['user_id', 'session_id', 'id', 'reference_transaction_id']
# Check for NaN values in the specified columns
def check_nan_in_columns(df, columns):
    # Iterate over the specified columns
    for col in columns:
        nan_count = df[col].isna().sum()  # Count NaN values in the column
        if nan_count > 0:
            print(f"Column '{col}' has {nan_count} NaN values.")
        else:
            print(f"Column '{col}' has no NaN values.")

# Example usage
# Assuming labelled_transaction_df is your DataFrame
check_nan_in_columns(df, nan_control_cols)


Column 'user_id' has no NaN values.
Column 'session_id' has no NaN values.
Column 'id' has no NaN values.
Column 'reference_transaction_id' has no NaN values.


In [102]:
# Assuming df is your combined dataset
# Ensure to drop 'user_id' and any other non-feature columns before proceeding
X = df.drop(columns=['user_current_state', 'user_id', 'reference_transaction_id', 'session_id'], errors='ignore')
y = df['user_current_state'].apply(lambda x: 1 if x == 'SUSPENDED' else 0)  # SUSPENDED=1, ACTIVE=0


In [103]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [104]:
# Define categorical and numeric columns again, ensuring 'user_id' or similar columns are not included
categorical_cols = X.select_dtypes(include=['object', 'bool']).columns.tolist()
numeric_cols = X.select_dtypes(include=['number']).columns.tolist()


In [105]:
# Imputer and encoders for numeric and categorical columns
numeric_transformer = SimpleImputer(strategy='mean')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [106]:
# Create a preprocessor to handle numeric and categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

## Model Training

In [107]:
# Train a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Create a pipeline with preprocessor and classifier
clf_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', clf)])

# Train the model
clf_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['phone_number_country_code',
                                                   'amount']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['country_of_residence',
                                                   'has_biometrics',
                                                   'reason_to_use_app',
                                                   'occupation', 'kyc_state',
                                                   'gender', 'date_of_birth',
                                                   'document_issuing_country',
                                                   'locale', 'screening_state',
                                                   'is_adverse_media_minor',
                                                   'is_pep', 'id', 'state',
                                                   'type', 'category_id',
                                                   'currency', 'direction',
                                                   'completed_at',
                                                   'created_at'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [108]:
# Make predictions
y_pred = clf_pipeline.predict(X_test)

In [109]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [110]:
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.95
Precision: 0.94
Recall: 0.90
F1 Score: 0.92


## Apply the Model to Unlabeled Data

In [111]:
# Call read data func
file_path = '../data/HomeTask _ [3] User Information Unlabelled.csv'
unlabelled_user_info_df = read_csv_file(file_path)

if unlabelled_user_info_df is not None:
    print(unlabelled_user_info_df.shape)  # Display dataframe shape if data was successfully loaded

File read successfully!
(300, 14)


In [112]:
# Drop duplicate rows from 'labelled_user_info_df'
unlabelled_user_info_df = unlabelled_user_info_df.drop_duplicates()
print(unlabelled_user_info_df.shape)

(282, 14)


In [113]:
# Call remove_nan_rows
unlabelled_cleaned_user_info_df = remove_nan_rows(unlabelled_user_info_df, 'user_id')
print(unlabelled_cleaned_user_info_df.shape)

(248, 14)


In [114]:
# Call read data func
file_path = '../data/HomeTask _ [4] TransactionData Unlabelled.csv'
unlabelled_transaction_df = read_csv_file(file_path)

if unlabelled_transaction_df is not None:
    print(unlabelled_transaction_df.shape)  # Display shape of dataframe if data was successfully loaded

File read successfully!
(9849, 10)


In [115]:
unlabelled_transaction_df.head()

,id,reference_transaction_id,user_id,state,type,amount,currency,direction,completed_at,created_at
0,a3c8bedb-312c-4155-a2f4-8883f58c03bc,568b3e92-9e58-4cd2-ad3b-28b8d8e7eb18,0eb40954-c525-4fbe-b691-dcdd0fb65208,FAILED,CARD_TOP_UP,50.0,EUR,IN,2024-08-25 22:42:42.423308 UTC,2024-08-25 22:42:38.977512 UTC
1,f1185255-0555-442a-9cad-eba75657a191,15ff6ae1-a71f-483a-985b-7c0c7a3c45cd,491576dd-e130-4330-af15-6252a30f0b3e,FAILED,CARD_TOP_UP,500.0,EUR,IN,2024-08-26 11:37:09.881278 UTC,2024-08-26 11:37:07.589171 UTC
2,68766cdf-555e-4dd3-a5cf-5e28d6f53a50,98005788-822e-4c00-a1bc-2bc10a4072a1,491576dd-e130-4330-af15-6252a30f0b3e,FAILED,CARD_TOP_UP,500.0,EUR,IN,2024-08-26 12:17:30.725341 UTC,2024-08-26 12:17:27.940281 UTC
3,f72eb354-1dd8-4570-8b75-54716fcd9050,9fdc0519-db17-45fc-b86b-6a740fbd42b4,d43eb4ce-1533-426f-852c-df0f43958922,FAILED,CARD_TOP_UP,500.0,EUR,IN,2024-08-26 09:05:37.213234 UTC,2024-08-26 09:05:33.016947 UTC
4,8977cd2a-f971-43ef-87fd-e7006e501845,30f7fe83-31c3-430c-a8ba-66cbe26811f9,d43eb4ce-1533-426f-852c-df0f43958922,FAILED,CARD_TOP_UP,500.0,EUR,IN,2024-08-26 09:03:07.343472 UTC,2024-08-26 09:03:00.671257 UTC


In [116]:
# Merge the date_of_birth column from labelled_user_info_df to unlabelled_cleaned_user_info_df based on 'user_id'
unlabelled_cleaned_user_info_df = unlabelled_cleaned_user_info_df.merge(
    cleaned_user_info_df[['user_id', 'date_of_birth']],  # Select only the 'user_id' and 'date_of_birth' columns
    on='user_id',  # Join on 'user_id'
    how='left'  # Perform a left join to retain all records from unlabeled_user_info_df
)

In [117]:
# Merge the category_id column from labelled_transaction_df to unlabelled_transaction_df based on 'reference_transaction_id'
unlabelled_transaction_df = unlabelled_transaction_df.merge(
    labelled_transaction_df[['reference_transaction_id', 'category_id']],  # Select only the 'reference_transaction_id' and 'category_id' columns
    on='reference_transaction_id',  # Join on 'reference_transaction_id'
    how='left'  # Perform a left join to retain all records from unlabeled_user_info_df
)

In [118]:
# Load the unlabeled data
# Assuming you have loaded the unlabeled datasets into unlabeled_transaction_df and unlabeled_user_info_df

# Merge the datasets
unlabeled_df = pd.merge(unlabelled_transaction_df, unlabelled_cleaned_user_info_df, on='user_id')

# Use the same feature engineering steps as the training data
unlabeled_df['transaction_count'] = unlabeled_df.groupby('user_id')['id'].transform('count')
unlabeled_df['total_amount'] = unlabeled_df.groupby('user_id')['amount'].transform('sum')
unlabeled_df['average_amount'] = unlabeled_df.groupby('user_id')['amount'].transform('mean')

# Drop any unnecessary columns
X_unlabeled = unlabeled_df.drop(columns=['user_id'])  # Remove ID column

# Make predictions on the unlabeled data
unlabeled_df['predicted_suspended'] = clf_pipeline.predict(X_unlabeled)

# Filter out customers predicted to be SUSPENDED
suspended_customers = unlabeled_df[unlabeled_df['predicted_suspended'] == 1]

# Generate a list of customers for off-boarding
offboarding_customers = suspended_customers[['user_id', 'transaction_count', 'total_amount', 'average_amount']]

print("Customers to be off-boarded based on predicted risk:")
print(offboarding_customers)


Customers to be off-boarded based on predicted risk:
                                   user_id  transaction_count  total_amount  \
63    85af9c86-a9d0-45c2-9891-7090d2f29cb5                 34      10702.00   
110   85af9c86-a9d0-45c2-9891-7090d2f29cb5                 34      10702.00   
203   39b6eca0-262e-44ec-a50b-4c25e92e4c15                 81      35267.19   
261   85af9c86-a9d0-45c2-9891-7090d2f29cb5                 34      10702.00   
312   39b6eca0-262e-44ec-a50b-4c25e92e4c15                 81      35267.19   
...                                    ...                ...           ...   
7354  85af9c86-a9d0-45c2-9891-7090d2f29cb5                 34      10702.00   
7411  0ed42672-70a4-4bda-8325-91d46cb62fed                  9      15150.00   
7416  a5a07f88-9c28-4670-82ff-8e164c85dbc0                 17      14733.96   
7541  39b6eca0-262e-44ec-a50b-4c25e92e4c15                 81      35267.19   
7543  39b6eca0-262e-44ec-a50b-4c25e92e4c15                 81      35267.19   

In [119]:
offboarding_customers.head()

,user_id,transaction_count,total_amount,average_amount
63,85af9c86-a9d0-45c2-9891-7090d2f29cb5,34,10702.00,314.764706
110,85af9c86-a9d0-45c2-9891-7090d2f29cb5,34,10702.00,314.764706
203,39b6eca0-262e-44ec-a50b-4c25e92e4c15,81,35267.19,435.397407
261,85af9c86-a9d0-45c2-9891-7090d2f29cb5,34,10702.00,314.764706
312,39b6eca0-262e-44ec-a50b-4c25e92e4c15,81,35267.19,435.397407


In [120]:
unlabeled_df.head()

,id,reference_transaction_id,user_id,state,type,amount,currency,direction,completed_at,created_at,...,document_issuing_country,locale,screening_state,is_adverse_media_minor,is_pep,date_of_birth,transaction_count,total_amount,average_amount,predicted_suspended
0,a3c8bedb-312c-4155-a2f4-8883f58c03bc,568b3e92-9e58-4cd2-ad3b-28b8d8e7eb18,0eb40954-c525-4fbe-b691-dcdd0fb65208,FAILED,CARD_TOP_UP,50.0,EUR,IN,2024-08-25 22:42:42.423308 UTC,2024-08-25 22:42:38.977512 UTC,...,FR,fr_FR,COMPLETED,False,False,NaN,103,12291.39,119.333883,0
1,f1185255-0555-442a-9cad-eba75657a191,15ff6ae1-a71f-483a-985b-7c0c7a3c45cd,491576dd-e130-4330-af15-6252a30f0b3e,FAILED,CARD_TOP_UP,500.0,EUR,IN,2024-08-26 11:37:09.881278 UTC,2024-08-26 11:37:07.589171 UTC,...,FR,fr_FR,COMPLETED,False,False,NaN,3,1014.99,338.330000,0
2,68766cdf-555e-4dd3-a5cf-5e28d6f53a50,98005788-822e-4c00-a1bc-2bc10a4072a1,491576dd-e130-4330-af15-6252a30f0b3e,FAILED,CARD_TOP_UP,500.0,EUR,IN,2024-08-26 12:17:30.725341 UTC,2024-08-26 12:17:27.940281 UTC,...,FR,fr_FR,COMPLETED,False,False,NaN,3,1014.99,338.330000,0
3,4fdf57d0-4af7-425a-ab40-a77b87c52c2f,8412e86b-149f-4ed5-8d38-b53a31cb700a,0eb40954-c525-4fbe-b691-dcdd0fb65208,FAILED,CARD_TOP_UP,50.0,EUR,IN,2024-08-25 13:28:44.837576 UTC,2024-08-25 13:28:41.739322 UTC,...,FR,fr_FR,COMPLETED,False,False,NaN,103,12291.39,119.333883,0
4,a27b29df-c898-456e-a959-df5e987df029,66262a17-d5f6-438e-b2e3-223d7f685dbd,0eb40954-c525-4fbe-b691-dcdd0fb65208,FAILED,CARD_TOP_UP,60.0,EUR,IN,2024-08-25 13:28:10.653343 UTC,2024-08-25 13:28:02.924041 UTC,...,FR,fr_FR,COMPLETED,False,False,NaN,103,12291.39,119.333883,0


In [121]:
unlabeled_df.predicted_suspended.value_counts()

predicted_suspended
0    7493
1     118
Name: count, dtype: int64